In [1]:
import torch
import string
import glob
import unicodedata
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from customlstm import CustomLSTM
from dataprep import Serialdata

In [2]:
class InferSerialData(Dataset):
    def __init__(self):
        self.all_chars = string.ascii_letters + " .,;'"
        self.num_char = len(self.all_chars)
        self.all_chars_dict = {v: k for k, v in enumerate(self.all_chars)}
        self.all_instances = []
    
    def unicodeToAscii(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in self.all_chars
        )
    
    def convert(self, lines):
        return_data = []
        for line in lines:
            temp = map(lambda a: self.all_chars_dict.get(a, self.num_char), self.unicodeToAscii(line))
            all_data = torch.tensor(list(temp))
            if all_data.size()[0] == 0:
                all_data = torch.tensor([57])
            return_data.append(all_data)
        return return_data

In [3]:
infd = InferSerialData()

In [4]:
input = infd.convert(["amuletos", "I like this", "thunder"])

In [5]:
# load lstm model for encoding
# sd = Serialdata('combined.tsv')

In [6]:
# sd.num_char

In [7]:
model = CustomLSTM(58, 256, num_layers=2).to('cuda')

In [8]:
model.load_state_dict(torch.load('lstm_model.pt'))

<All keys matched successfully>

In [9]:
x, y = model(torch.tensor(input).unsqueeze(0).to('cuda'))

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
p = torch.tensor(infd.convert(",")).unsqueeze(0)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

In [ ]:
input_padded = pad_sequence(input, batch_first=True, padding_value=57)

In [ ]:
input

In [ ]:
list(model.named_children())[0][1](input_padded.to('cuda'))

In [ ]:
y[0,:, :].shape

In [ ]:
torch.mean(y, dim=0, keepdim=True).shape

In [ ]:
k = torch.randn([1,1,768]).to('cuda')

In [ ]:
torch.cat([torch.mean(y, dim=0, keepdim=True),k], dim=2).shape

In [ ]:
list(model.named_children())[0][1](torch.tensor(57).cuda())

In [ ]:
torch.mean(y, dim=0, keepdim=True)

In [ ]:
torch.stack([torch.mean(y, dim=0, keepdim=True),torch.mean(y, dim=0, keepdim=True)], dim=2).shape

In [ ]:
input_padded.to('cuda').shape

In [ ]:
_, t = model(input_padded.to('cuda'))

In [ ]:
t.mean(dim=0, keepdim=True).squeeze(0).shape

In [ ]:
z = torch.zeros([5,256])

In [ ]:
z[:3,:] = t.mean(dim=0, keepdim=True).squeeze(0)

In [ ]:
torch.cat([torch.randn([32, 38, 768]), torch.randn([32, 38, 256])], dim=-1).shape

In [20]:
target = torch.randn([32,38, 73])

In [11]:
inp = torch.randn([32, 38])

In [40]:
inp = torch.randint(0,65, [32,38])

In [41]:
from kornia.losses import FocalLoss

In [42]:
fl = FocalLoss(alpha=2, reduction='mean')

In [43]:
target.permute(0,2,1).shape

torch.Size([32, 73, 38])

In [44]:
target.shape, inp.shape

(torch.Size([32, 38, 73]), torch.Size([32, 38]))

In [45]:
fl(target.permute(0,2,1), inp)

tensor(9.3483)